In [1]:
library(Matrix)
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(viridis))
library(glmGamPoi)
library(RColorBrewer)
options(future.globals.maxSize = 10000 * 1024^2)
setwd("/share/crsp/lab/seyedam/share/enc4_mouse/snrna/")

Warning message in system("timedatectl", intern = TRUE):
"running command 'timedatectl' had status 1"


In [2]:
ref = readMM("ref/external_data/brain_atlas_1000_per_subtype_250734.mtx")

In [3]:
genes = read.delim("ref/external_data/brain_atlas_1000_per_subtype_250734_genes.tsv",header=F)

In [4]:
barcodes = read.delim("ref/external_data/brain_atlas_1000_per_subtype_250734_barcodes.tsv",header=F)

In [5]:
rownames(ref) = genes$V1
colnames(ref) = barcodes$V1

In [6]:
ref = ref[1:(nrow(ref)-129),]

In [7]:
meta = read.csv("ref/external_data/brain_atlas_metadata_1000_per_subtype_250734.csv")

In [8]:
ref_obj = CreateSeuratObject(counts = ref, min.cells = 0, min.features = 0)
ref_obj[["percent.mt"]] = PercentageFeatureSet(ref_obj, pattern = "^mt-")
ref_obj@meta.data = cbind(ref_obj@meta.data,meta)



In [9]:
ref_obj

An object of class Seurat 
31053 features across 250734 samples within 1 assay 
Active assay: RNA (31053 features, 0 variable features)

In [ ]:
ref_obj <- SCTransform(ref_obj, vars.to.regress = c("nFeature_RNA","percent.mt"),
                  method = "glmGamPoi", conserve.memory = TRUE)

Calculating cell attributes from input UMI matrix: log_umi



In [ ]:

ref_obj <- RunPCA(ref_obj, verbose = T, assay = "SCT")


In [ ]:
ref_obj <- RunUMAP(ref_obj, reduction = "pca", dims = 1:30,verbose = F)
ref_obj <- FindNeighbors(ref_obj, reduction = "pca", dims = 1:30,verbose = F)


In [ ]:
ref_obj <- FindClusters(ref_obj,resolution = 0.25,verbose = F)


In [ ]:
options(repr.plot.width = 20,repr.plot.height=10)
nclusters = length(unique(ref_obj$cell_type_alias_label))

DimPlot(ref_obj, reduction = "umap", group.by = "cell_type_alias_label",
        label = TRUE, label.size = 6, repel = TRUE,cols = colorRampPalette(brewer.pal(9,"Set1"))(nclusters))  & NoLegend()

